In [2]:
import json
from pathlib import Path

# ---------------- CONFIG ----------------
SDXL_DIR = Path("train/lora")
LORA_DIR = Path("train/lora_2X_resumed")
OUTPUT_FILE = "comparison_report_2X_resumed.html"
IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".webp"}
# ---------------------------------------

# Load prompt data
with open("samples.json", "r", encoding="utf-8") as f:
    prompts = json.load(f)

# Collect & sort images
sdxl_images = sorted(p for p in SDXL_DIR.iterdir() if p.suffix.lower() in IMAGE_EXTS)
lora_images = sorted(p for p in LORA_DIR.iterdir() if p.suffix.lower() in IMAGE_EXTS)

count = min(len(prompts), len(sdxl_images), len(lora_images))
print(f"Using {count} comparisons")

html = """
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>SDXL vs LoRA Comparison</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: #111;
    color: #eee;
    padding: 20px;
}

h1 { margin-bottom: 20px; }

.toolbar {
    position: fixed;
    top: 20px;
    right: 20px;
    z-index: 1000;
}

.btn {
    background: #2d7dff;
    color: white;
    border: none;
    padding: 10px 14px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
}

.row {
    margin-bottom: 60px;
    page-break-after: always;
}

.prompt {
    font-size: 14px;
    color: #ccc;
    margin-bottom: 8px;
}

.meta {
    font-size: 12px;
    color: #888;
    margin-bottom: 12px;
}

.images {
    display: flex;
    gap: 20px;
}

.images img {
    max-width: 512px;
    border-radius: 8px;
    border: 1px solid #333;
    cursor: pointer;
}

/* ---------- LIGHTBOX ---------- */
.lightbox {
    display: none;
    position: fixed;
    inset: 0;
    background: rgba(0,0,0,0.96);
    z-index: 2000;
}

.lightbox.active {
    display: flex;
    justify-content: center;
    align-items: center;
}

.lightbox-content {
    position: relative;
    width: 100%;
    height: 100%;
    padding: 30px;
    display: flex;
    flex-direction: column;
}

.lightbox-header {
    color: #aaa;
    font-size: 14px;
    margin-bottom: 10px;
    text-align: center;
}

.lightbox-images {
    flex: 1;
    display: flex;
    justify-content: center;
    align-items: center;
}

.lightbox-images img {
    max-height: 90vh;
    max-width: 95vw;
    border-radius: 12px;
    border: 1px solid #333;
}

/* Elegant close */
.lightbox-close {
    position: absolute;
    top: 18px;
    right: 22px;
    width: 36px;
    height: 36px;
    font-size: 26px;
    line-height: 36px;
    text-align: center;
    color: #ddd;
    cursor: pointer;
    transition: all 0.2s ease;
}

.lightbox-close:hover {
    color: white;
    transform: rotate(90deg) scale(1.1);
}

/* ---------- PRINT ---------- */
@media print {
    body {
        background: white;
        color: black;
    }
    .toolbar,
    .lightbox {
        display: none !important;
    }
}
</style>

<script>
window.ALL_COMPARISONS = [];

let currentPair = 0;
let currentSide = 0; // 0 = SDXL, 1 = LoRA

function openLightbox(index, side) {
    currentPair = index;
    currentSide = side;
    updateLightbox();
    document.getElementById("lightbox").classList.add("active");
}

function updateLightbox() {
    const pair = window.ALL_COMPARISONS[currentPair];
    const img = currentSide === 0 ? pair.sdxl : pair.lora;
    document.getElementById("lightboxImage").src = img;
    document.getElementById("lightboxHeader").innerText =
        `Prompt ${currentPair + 1} — ${currentSide === 0 ? "SDXL Base" : "LoRA SDXL"}`;
}

function next() {
    if (currentSide === 0) {
        currentSide = 1;
    } else {
        currentSide = 0;
        currentPair = (currentPair + 1) % window.ALL_COMPARISONS.length;
    }
    updateLightbox();
}

function prev() {
    if (currentSide === 1) {
        currentSide = 0;
    } else {
        currentSide = 1;
        currentPair = (currentPair - 1 + window.ALL_COMPARISONS.length) % window.ALL_COMPARISONS.length;
    }
    updateLightbox();
}

function closeLightbox() {
    document.getElementById("lightbox").classList.remove("active");
}

document.addEventListener("keydown", e => {
    const lb = document.getElementById("lightbox");
    if (!lb.classList.contains("active")) return;

    if (e.key === "Escape") closeLightbox();
    if (e.key === "ArrowRight") next();
    if (e.key === "ArrowLeft") prev();
});

function outsideClick(e) {
    if (e.target.id === "lightbox") closeLightbox();
}

function exportPDF() {
    window.print();
}
</script>
</head>

<body>

<div class="toolbar">
    <button class="btn" onclick="exportPDF()">📄 Export PDF</button>
</div>

<h1>SDXL Base vs LoRA SDXL Comparison</h1>
"""

# ---------- CONTENT ----------
for i in range(count):
    item = prompts[i]

    prompt = item.get("prompt", "")
    seed = item.get("seed")
    steps = item.get("diffusion_steps")
    cfg = item.get("cfg_scale")
    scheduler = item.get("noise_scheduler")

    sdxl = sdxl_images[i]
    lora = lora_images[i]

    html += f"""
<script>
window.ALL_COMPARISONS.push({{
    sdxl: "{sdxl}",
    lora: "{lora}"
}});
</script>

<div class="row">
    <div class="prompt"><b>Prompt:</b> {prompt}</div>

    <div class="images">
        <div>
            <img src="{sdxl}" onclick="openLightbox({i}, 0)">
        </div>
        <div>
            <img src="{lora}" onclick="openLightbox({i}, 1)">
        </div>
    </div>
</div>
"""

# ---------- LIGHTBOX ----------
html += """
<div id="lightbox" class="lightbox" onclick="outsideClick(event)">
    <div class="lightbox-content" onclick="event.stopPropagation()">
        <div class="lightbox-close" onclick="closeLightbox()">✕</div>
        <div class="lightbox-header" id="lightboxHeader"></div>
        <div class="lightbox-images">
            <img id="lightboxImage">
        </div>
    </div>
</div>

</body>
</html>
"""

# Write output
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Comparison report generated: {OUTPUT_FILE}")


Using 9 comparisons
✅ Comparison report generated: comparison_report_2X_resumed.html


In [ ]:
# <div class="meta">
#     Seed: {seed} | Steps: {steps} | CFG: {cfg} | Scheduler: {scheduler}
# </div>